In [7]:
# import internal .py modules
import file_path_management as fpath
import public_library as plib

In [8]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re

In [9]:
# get doi from url
def url2doi(url, source):
    url = str(url).strip()

    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    
    error_label = 0
    while(error_label == 0):
        try:
            driver = webdriver.Firefox(options, service=Service(GeckoDriverManager().install()))
            driver.get(url)

            # WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//p[text()='Consent']"))).click()

            # try:
            #     WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#formInput"))).send_keys(str(doi).strip())
            # except TimeoutException:
            #     print("Waiting for clicking consent timeout")
            # try:
            #     # WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[2]/form/button"))).click()
            #     WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//button[text()="Get PMID"]'))).click()
            # except TimeoutException:
            #     print("Waiting for clicking button timeout")
            
            error_label = 1
        except:
            print("URL to DOI transformation failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0

    try:
        my_elem = driver.find_elements(By.XPATH, "//*[contains(text(), 'doi.org') or contains(text(), 'DOI')]")
        doi = ""
        doi = str(my_elem.find_element(By.XPATH, "i").get_attribute("innerHTML"))
    except:
        doi = np.nan
    driver.quit()
    return doi
# --------------------start of test code--------------------
# url = "https://www.tandfonline.com/doi/abs/10.1080/01616412.1985.11739692"
# doi = url2doi(url)
# print(doi)
# ---------------------end of test code---------------------

In [10]:
def get_final_redirected_url(url):
    response = requests.get(url, headers = plib.headers)
    
    while(response.status_code != 200):
        if response.status_code == 403:
            final_url = url
            break
        elif response.status_code == 404:
            final_url = np.nan
            break
        else:
            print(response.status_code)     
            # sleep for 5 minutes
            time.sleep(300)
            response = requests.get(url, headers = plib.headers)
    if response.status_code == 200:
        final_url = response.url
    # history = response.history
    return final_url

In [11]:
def preprocess_google_shcolar_step1(source_path, output_path, start, end):
    print("Starting merging search results from Google Scholar...")

    df = pd.read_csv(source_path, header=None, sep=',')
    df.columns = ["title", "url", "url_type", "full_text_url", "full_text_type", "full_text_source"]

    for ind in range(start, end):
        # df["url_type"][ind]: {'[CITATION][C]', '[PDF][PDF]', '[BOOK][B]', nan, '[HTML][HTML]'}
        # if df["url_type"][ind] == "[CITATION][C]" or "[BOOK][B]", skip this row
        if (df["url_type"][ind] == "[CITATION][C]") or (df["url_type"][ind] == "[BOOK][B]"):
            continue
        
        # if url or title doesn't exsit AND full_text_url doesn't exist
        if (df["url"][ind] != df["url"][ind]) or (df["title"][ind] != df["title"][ind]):
            continue
        
        # now every row has at least title and url, and the url_text = {"[PDF][PDF]", nan, "[HTML][HTML]"}
        if df["url_type"][ind] == "[PDF][PDF]":
            # full_text_type = {'[HTML]', nan, '[PDF]', 'UB'}
            if df["full_text_type"][ind] != "[HTML]":
                full_text_url = np.nan
                full_text_source = np.nan
            else:
                link = str(df["full_text_url"][ind]).strip()
                full_text_url = get_final_redirected_url(link)
                if full_text_url == full_text_url:
                    full_text_source = full_text_url.split("://")[1].split("/")[0]
                else:
                    full_text_source = np.nan
            # get pdf_url, pdf_source
            link = str(df["url"][ind]).strip()
            pdf_url = get_final_redirected_url(link)
            if pdf_url == pdf_url:
                pdf_source = pdf_url.split("://")[1].split("/")[0]
            else:
                pdf_source = np.nan
        else: # df["url_type"][ind] == nan or '[HTML][HTML]'
            link = str(df["url"][ind]).strip()
            full_text_url = get_final_redirected_url(link)
            if full_text_url == full_text_url:
                full_text_source = full_text_url.split("://")[1].split("/")[0]
            else:
                full_text_source = np.nan
            # get pdf_url, pdf_source
            # full_text_type = {'[HTML]', nan, '[PDF]', 'UB'}
            if df["full_text_type"][ind] == "[PDF]":
                link = str(df["full_text_url"][ind]).strip()
                pdf_url = get_final_redirected_url(link)
                if pdf_url == pdf_url:
                    pdf_source = pdf_url.split("://")[1].split("/")[0]
                else:
                    pdf_source = np.nan
            else:
                pdf_url = np.nan
                pdf_source = np.nan
        
        columns = ["Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]
        row = {
            "Title": [str(df["title"][ind]).strip()],
            "full_text_url": [full_text_url],
            "full_text_source": [full_text_source],
            "pdf_url": [pdf_url],
            "pdf_source": [pdf_source]
        }
        # print(row)

        if not plib.add_row_to_csv(output_path, row, columns):
            print("Error detected when adding a row to csv!")
        
        print(ind)
# --------------------start of test code--------------------
source_path = fpath.poten_litera_gs
output_path = fpath.poten_litera_gs_processed_step1
# plib.clear_file(output_path)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# df = pd.read_csv(source_path, header=None, sep=',')
# df.columns = ["title", "url", "url_type", "full_text_url", "full_text_type", "full_text_source"]
# print(df.head(3))
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# url_type = set(df['url_type'].tolist())
# print(url_type)
# # {'[CITATION][C]', '[PDF][PDF]', '[BOOK][B]', nan, '[HTML][HTML]'}
# full_text_type = set(df['full_text_type'].tolist())
# print(full_text_type)
# # {nan, 'UB', '[HTML]', '[PDF]'}
# full_text_source = set(df['full_text_source'].tolist())
# print(full_text_source)
# # {'ahajournals.org', 'lww.com', 'springer.com', 'academia.edu', 'plos.org', 'ieee.org', 'nature.com', 
# # 'mdpi.com', 'jpn.ca', 'uottawa.ca', nan, 'northwestern.edu', 'bmj.com', 'ekja.org', 'RWTH-Link', 'wiley.com', 
# # 'escholarship.org', 'nyu.edu', 'frontiersin.org', 'sciencedirect.com', 'eneuro.org', 'jneurosci.org', 
# # 'royalsocietypublishing.org', 'karger.com', 'harvard.edu', 'annualreviews.org', 'mcgill.ca', 
# # 'elifesciences.org', 'mirasmart.com', 'duke.edu', 'ucdavis.edu', 'physiology.org', 'cell.com', 
# # 'wustl.edu', 'epfl.ch', 'udc.es', 'psychiatryonline.org', 'jst.go.jp', 'core.ac.uk', 'rero.ch', 
# # 'zsp.com.pk', 'sagepub.com', 'europepmc.org', 'tandfonline.com', 'asahq.org', 'sonar.ch', 'koreamed.org', 
# # 'oup.com', 'science.org', 'scholarpedia.org', 'psu.edu', 'jordanbpeterson.com', 'pnas.org', 'uzh.ch', 'biorxiv.org', 
# # 'biomedcentral.com', 'umich.edu', 'ahuman.org', 'researchgate.net', 'ijpp.com', 'unav.edu', 'nih.gov', 'bu.edu'}
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# # ["title", "url", "url_type", "full_text_url", "full_text_type", "full_text_source"]
# print(df["title"].isnull().any().any())
# print(df["url"].isnull().any().any())
# print(df["url_type"].isnull().any().any())
# print(df["full_text_url"].isnull().any().any())
# print(df["full_text_type"].isnull().any().any())
# print(df["full_text_source"].isnull().any().any())
# # True, True, True, True, True, True
# # title, url, url_type, full_text_url, full_text_type, full_text_source contain np.nan
# # we need to fill in what are missing
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# preprocess_google_shcolar_step1(source_path, output_path, 0, 1000)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
df = pd.read_csv(output_path, header=None, sep=',')
print(df.head(3))
# ---------------------end of test code---------------------

                                                   0  \
0  Effect of attentive fixation in macaque thalam...   
1  The thalamus of the Macaca, mulatta. An atlas ...   
2  Distribution of the dopamine innervation in th...   

                                                   1                        2  \
0  https://journals.physiology.org/doi/abs/10.115...  journals.physiology.org   
1  https://www.cabdirect.org/cabdirect/welcome/?t...        www.cabdirect.org   
2  https://www.sciencedirect.com/science/article/...    www.sciencedirect.com   

                                                   3                        4  
0  https://journals.physiology.org/doi/pdf/10.115...  journals.physiology.org  
1                                                NaN                      NaN  
2                                                NaN                      NaN  


In [12]:
def preprocess_google_shcolar_step2(source_path, output_path, columns, start, end):
    print("Starting merging search results from Google Scholar...")

    df = pd.read_csv(source_path, header=None, sep=',')
    df.columns = ["Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]

    for ind in range(start, end):
        # get doi from url
        url = str(df["full_text_url"][ind]).strip()
        source = str(df["full_text_url"][ind]).strip()
        doi = url2doi(url, source)

        # get pmid from DOI
        if doi == doi: # there's doi
            pmid = plib.doi2pmid(doi)
        else: # doi not found
            pmid = np.nan

        # get pmcid
        if pmid == pmid: # there's pmid
            url = "https://pubmed.ncbi.nlm.nih.gov/" + pmid + "/"
            # proxies = plib.get_proxies()
            soup = plib.request_webpage(url)
            # print(soup)
            try:
                pmcid = soup.find_all("span", {"class": "identifier pmc"})[0].find_all("a", {"class": "id-link"})[0].get_text().strip()
            except: # pmcid not found
                pmcid = np.nan
        else: # pmid not found
            pmcid = np.nan
        # print(pmcid)

        # columns = ["DOI", "PMID", "PMCID", "Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]
        row = {
            "DOI": [doi],
            "PMID": [pmid],
            "PMCID": [pmcid],
            "Title": [str(df["Title"][ind]).strip()],
            "full_text_url": [str(df["full_text_url"][ind]).strip()],
            "full_text_source": [str(df["full_text_source"][ind]).strip()],
            "pdf_url": [str(df["pdf_url"][ind]).strip()],
            "pdf_source": [str(df["pdf_source"][ind]).strip()],
        }
        # print(row)

        if not plib.add_row_to_csv(output_path, row, columns):
            print("Error detected when adding a row to csv!")
        
        print(ind)
# --------------------start of test code--------------------
source_path = fpath.poten_litera_gs_processed_step1
output_path = fpath.poten_litera_gs_processed_step2
plib.clear_file(output_path)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
df = pd.read_csv(source_path, header=None, sep=',')
df.columns = ["Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]
print(df.head(3))
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
full_text_source = set(df['full_text_source'].tolist())
print(full_text_source)
# 
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# ["Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]
print(df["Title"].isnull().any().any())
print(df["full_text_url"].isnull().any().any())
print(df["full_text_source"].isnull().any().any())
print(df["pdf_url"].isnull().any().any())
print(df["pdf_source"].isnull().any().any())
# 
# contain np.nan
# we need to fill in what are missing
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# preprocess_google_shcolar_step2(source_path, output_path, columns, 0, 1000)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# df = pd.read_csv(output_path, header=None, sep=',')
# print(df.head(3))
# ---------------------end of test code---------------------

                                               Title  \
0  Effect of attentive fixation in macaque thalam...   
1  The thalamus of the Macaca, mulatta. An atlas ...   
2  Distribution of the dopamine innervation in th...   

                                       full_text_url         full_text_source  \
0  https://journals.physiology.org/doi/abs/10.115...  journals.physiology.org   
1  https://www.cabdirect.org/cabdirect/welcome/?t...        www.cabdirect.org   
2  https://www.sciencedirect.com/science/article/...    www.sciencedirect.com   

                                             pdf_url               pdf_source  
0  https://journals.physiology.org/doi/pdf/10.115...  journals.physiology.org  
1                                                NaN                      NaN  
2                                                NaN                      NaN  
{'www.degruyter.com', 'www.cabdirect.org', 'www.liebertpub.com', 'www.cell.com', 'onlinelibrary.wiley.com', nan, 'bmcneurosci.biom